In [1]:
%pip install spark-nlp==6.0.2 numpy pandas matplotlib seaborn setuptools wordcloud pypdf2 contractions unidecode
%pip install --upgrade setuptools

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import findspark
findspark.init('./spark/spark-3.5.1-bin-hadoop3')
from pyspark import *
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DateType, TimestampType, LongType
from pyspark.sql.types import ArrayType, DoubleType, BooleanType, DecimalType
from pyspark.sql.functions import regexp_extract, split, from_unixtime, col, avg, min, max
from pyspark.sql.functions import grouping_id, window, explode, to_json, from_json
from pyspark.sql.functions import udf, lit, current_timestamp, current_date, date_format
from pyspark.ml import PipelineModel, Pipeline
import PyPDF2
import os

In [3]:
spark = SparkSession.builder \
    .appName("nlpdemo") \
    .config("spark.jars.packages", 
            ",".join([
                "com.johnsnowlabs.nlp:spark-nlp_2.12:6.0.2"
            ])) \
    .getOrCreate()

25/07/27 15:57:13 WARN Utils: Your hostname, Joses-MacBook-Air-2.local resolves to a loopback address: 127.0.0.1; using 10.105.113.167 instead (on interface en0)
25/07/27 15:57:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/joseaguilar/.ivy2/cache
The jars for the packages stored in: /Users/joseaguilar/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a08e6fc2-cf0f-480a-8dcf-1851787bd21c;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;6.0.2 in central
	found com.typesafe#config;1.4.2 in central


:: loading settings :: url = jar:file:/Users/joseaguilar/Documents/Development/spark/spark-3.5.1-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-s3;1.12.500 in central
	found com.amazonaws#aws-java-sdk-kms;1.12.500 in central
	found com.amazonaws#aws-java-sdk-core;1.12.500 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in local-m2-cache
	found org.apache.httpcomponents#httpclient;4.5.13 in local-m2-cache
	found org.apache.httpcomponents#httpcore;4.4.13 in local-m2-cache
	found software.amazon.ion#ion-java;1.0.2 in central
	found joda-time#joda-time;2.8.1 in central
	found com.amazonaws#jmespath-java;1.12.500 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in central
	found com.google.cloud#goo

In [4]:
import sparknlp
from sparknlp.base import DocumentAssembler, Finisher
from sparknlp.annotator import Tokenizer, Normalizer

print(sparknlp.version())
spark = sparknlp.start()

6.0.2


25/07/27 15:57:19 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [5]:
oraciones = [
    "It's essential to drink coffee at a café and to keep one’s emotions in check while analyzing complex, real-time data!",
    "I'm going to the store because I can't wait to buy some groceries.",
    "El número pi es aproximadamente 3,1416. Ya ahora saben."
]
df_oraciones = spark.createDataFrame([(s,) for s in oraciones], ["texto"])
df_oraciones.show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------+
|texto                                                                                                                |
+---------------------------------------------------------------------------------------------------------------------+
|It's essential to drink coffee at a café and to keep one’s emotions in check while analyzing complex, real-time data!|
|I'm going to the store because I can't wait to buy some groceries.                                                   |
|El número pi es aproximadamente 3,1416. Ya ahora saben.                                                              |
+---------------------------------------------------------------------------------------------------------------------+



In [6]:
import contractions
from pyspark.sql.functions import col, udf

# Definir una UDF para expandir contracciones en inglés
expand_contr = udf(lambda text: contractions.fix(text))
df_expanded = df_oraciones.withColumn("texto_exp", expand_contr(col("texto")))
df_expanded.select("texto", "texto_exp").show(truncate=False)


+---------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------+
|texto                                                                                                                |texto_exp                                                                                                             |
+---------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------+
|It's essential to drink coffee at a café and to keep one’s emotions in check while analyzing complex, real-time data!|It is essential to drink coffee at a café and to keep one’s emotions in check while analyzing complex, real-time data!|
|I'm going to the store because I can't wait

In [8]:
from sparknlp.annotator import Normalizer

normalizer_no_punc = Normalizer()\
    .setInputCols(["tokenized"]).setOutputCol("document_norm")\
    .setLowercase(True)\
    .setCleanupPatterns(["[^\\w\\sáéíóúÁÉÍÓÚüÜñÑ]"])  # conservar letras, dígitos y espacios

pipeline_norm = Pipeline(stages=[DocumentAssembler().setInputCol("texto_exp").setOutputCol("document"),
                                 Tokenizer().setInputCols(["document"]).setOutputCol("tokenized"),
                                 normalizer_no_punc,
                                 Finisher().setInputCols(["document_norm"]).setOutputCols(["texto_limpio"])])
df_clean = pipeline_norm.fit(df_expanded).transform(df_expanded)
df_clean.select("texto_exp", "texto_limpio").show(truncate=False)


+----------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------+
|texto_exp                                                                                                             |texto_limpio                                                                                                                            |
+----------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------+
|It is essential to drink coffee at a café and to keep one’s emotions in check while analyzing complex, real-time data!|[it, is, essential, to, drink, coffee, at, a, café, and, to, keep, ones, emotions, in, check, while, analy

In [10]:
import unidecode
remove_accents = udf(lambda text: unidecode.unidecode(text))
df_no_accents = df_clean.withColumn("texto_sin_acentos", remove_accents(col("texto_exp")))
df_no_accents.select("texto_exp", "texto_sin_acentos").show(truncate=False)


+----------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------+
|texto_exp                                                                                                             |texto_sin_acentos                                                                                                     |
+----------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------+
|It is essential to drink coffee at a café and to keep one’s emotions in check while analyzing complex, real-time data!|It is essential to drink coffee at a cafe and to keep one's emotions in check while analyzing complex, real-time data!|
|I am going to the store because I canno

In [17]:
from sparknlp.base import DocumentAssembler, Finisher
from sparknlp.annotator import Tokenizer, StopWordsCleaner
from pyspark.ml import Pipeline

# 1️⃣ DocumentAssembler: convierte texto plano en anotación "document"
document_assembler = (
    DocumentAssembler()
    .setInputCol("texto_sin_acentos")   # <- tu columna con texto limpio
    .setOutputCol("document")           # ← salida que usará el Tokenizer
)

# 2️⃣ Tokenizer: parte el documento en palabras
tokenizer = (
    Tokenizer()
    .setInputCols(["document"])         # <- ahora sí, recibe "document"
    .setOutputCol("token")
)

# 3️⃣ StopWordsCleaner: elimina stop‑words
stopwords_cleaner = (
    StopWordsCleaner()
    .setInputCols(["token"])
    .setOutputCol("token_sin_stop")
    .setCaseSensitive(False)
    .setStopWords(["y", "de", "la", "the", "and", "is", "es", "porque", "to", "am", "el", "in", "some"])
)

# 4️⃣ Finisher: extrae los tokens finales como lista de strings
finisher = (
    Finisher()
    .setInputCols(["token_sin_stop"])
    .setOutputCols(["tokens_final"])
)

# 5️⃣ Pipeline completo
pipeline_stop = Pipeline(stages=[
    document_assembler,
    tokenizer,
    stopwords_cleaner,
    finisher
])

df_tokens_final = pipeline_stop.fit(df_no_accents).transform(df_no_accents)

df_tokens_final.select("texto_limpio", "tokens_final").show(truncate=False)


+----------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------+
|texto_limpio                                                                                                                            |tokens_final                                                                                                               |
+----------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------+
|[it, is, essential, to, drink, coffee, at, a, café, and, to, keep, ones, emotions, in, check, while, analyzing, complex, realtime, data]|[It, essential, drink, coffee, at, a, cafe, keep, one's, emotions, check,